# Timer

In [7]:
from time import sleep
from datetime import datetime as dt

start_time = "10.12.2022, 22:25"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [8]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'LinearReg': {'timeperiod': [4, 6, 8, 10, 12], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 15m/1h, trade type is buy
Number of combinations is 100


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [11:28<00:00,  6.88s/it]


# Check local statistics

In [9]:
# 15m/1h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
99,PriceChange_LinearReg,20,12,0,1.2788,1.4447,1.4529,1.5434,1.5860,1.6022,1.5917,1.5602,1.5480,1.5903,1.5878,1.5926,1.5887,1.5938,1.5998,1.5798,1.5713,1.5522,1.5202,1.4948,1.4881,1.4809,1.4641,1.4755,0.72,0.82,0.51,0.37,0.38,0.36,0.33,0.26,0.24,0.24,0.12,0.12,0.00,0.00,-0.14,-0.32,-0.32,-0.54,-0.73,-0.77,-0.85,-0.97,-1.04,-1.16,328,1.532825,-0.098750,174.766600,-32.390000
95,PriceChange_LinearReg,20,4,0,1.3657,1.6012,1.6432,1.7427,1.7660,1.7793,1.7871,1.7220,1.7009,1.7430,1.7368,1.7470,1.7471,1.7447,1.7450,1.7252,1.6903,1.6606,1.6328,1.6047,1.5941,1.5870,1.5637,1.5686,0.71,0.91,0.53,0.54,0.57,0.37,0.36,0.37,0.27,0.25,0.11,0.09,0.00,0.00,-0.02,-0.22,-0.28,-0.46,-0.62,-0.71,-0.72,-0.80,-0.89,-1.14,250,1.674946,-0.032500,168.736458,-8.125000
96,PriceChange_LinearReg,20,6,0,1.3313,1.5240,1.5573,1.6469,1.6614,1.6789,1.6778,1.6245,1.6097,1.6480,1.6465,1.6511,1.6499,1.6516,1.6594,1.6417,1.6119,1.5945,1.5670,1.5408,1.5363,1.5279,1.5085,1.5218,0.71,0.86,0.50,0.37,0.49,0.35,0.34,0.29,0.25,0.24,0.06,0.00,0.00,0.00,-0.11,-0.25,-0.28,-0.54,-0.73,-0.73,-0.80,-0.84,-0.97,-1.13,283,1.594529,-0.080000,168.251754,-22.640000
80,PriceChange_LinearReg,17,4,0,1.4088,1.6658,1.7414,1.8332,1.8388,1.8535,1.8787,1.8019,1.7418,1.7883,1.7849,1.7911,1.7865,1.7832,1.7832,1.7602,1.7214,1.6962,1.6692,1.6404,1.6238,1.6180,1.5947,1.5945,0.71,0.98,0.56,0.60,0.60,0.53,0.44,0.40,0.37,0.29,0.10,0.09,0.00,0.00,0.00,-0.22,-0.27,-0.46,-0.64,-0.75,-0.76,-0.90,-0.91,-1.16,224,1.724979,-0.016667,162.395333,-3.733333
91,PriceChange_LinearReg,19,6,0,1.3175,1.5148,1.5505,1.6199,1.6443,1.6647,1.6618,1.6087,1.5948,1.6338,1.6376,1.6404,1.6378,1.6404,1.6487,1.6326,1.6038,1.5916,1.5648,1.5385,1.5347,1.5261,1.5065,1.5202,0.71,0.87,0.48,0.35,0.49,0.34,0.33,0.29,0.25,0.24,0.05,0.00,0.00,0.00,-0.10,-0.24,-0.28,-0.52,-0.69,-0.73,-0.78,-0.82,-0.95,-1.13,276,1.584771,-0.076667,161.396750,-21.160000
90,PriceChange_LinearReg,19,4,0,1.3522,1.5892,1.6332,1.7113,1.7463,1.7629,1.7689,1.6992,1.6793,1.7226,1.7226,1.7314,1.7304,1.7290,1.7297,1.7118,1.6779,1.6540,1.6272,1.5990,1.5890,1.5818,1.5609,1.5661,0.70,0.93,0.51,0.50,0.57,0.36,0.34,0.37,0.27,0.22,0.08,0.04,0.00,0.00,-0.02,-0.22,-0.28,-0.46,-0.62,-0.71,-0.72,-0.80,-0.89,-1.14,242,1.661496,-0.040417,160.081992,-9.780833
81,PriceChange_LinearReg,17,6,0,1.3691,1.5711,1.6346,1.7166,1.7143,1.7342,1.7463,1.6840,1.6341,1.6725,1.6716,1.6740,1.6695,1.6715,1.6819,1.6616,1.6282,1.6173,1.5922,1.5664,1.5571,1.5509,1.5319,1.5416,0.72,0.95,0.53,0.40,0.53,0.41,0.37,0.34,0.27,0.27,0.00,0.00,0.00,0.00,-0.12,-0.27,-0.29,-0.54,-0.75,-0.77,-0.85,-0.97,-1.01,-1.14,254,1.628854,-0.080000,159.728958,-20.320000
94,PriceChange_LinearReg,19,12,0,1.2637,1.4317,1.4467,1.5175,1.5383,1.5589,1.5496,1.5162,1.5061,1.5510,1.5539,1.5577,1.5533,1.5600,1.5653,1.5454,1.5389,1.5245,1.4934,1.4683,1.4627,1.4558,1.4413,1.4535,0.72,0.81,0.47,0.34,0.34,0.34,0.31,0.23,0.18,0.22,0.09,0.04,0.00,-0.11,-0.21,-0.40,-0.36,-0.63,-0.78,-0.79,-0.91,-1.02,-1.10,-1.18,316,1.502237,-0.141667,158.707050,-44.766667
84,PriceChange_LinearReg,17,12,0,1.2961,1.4648,1.4952,1.5759,1.5868,1.6081,1.6169,1.5816,1.5407,1.5852,1.5839,1.5872,1.5808,1.5871,1.5956,1.5736,1.5648,1.5513,1.5212,1.4957,

In [6]:
# 15m/4h
stat['e_ratio_avg'] = stat[[f'e_ratio_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['e_ratio_rank'] = (stat['e_ratio_avg'] - 1) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('e_ratio_rank', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,e_ratio_1,e_ratio_2,e_ratio_3,e_ratio_4,e_ratio_5,e_ratio_6,e_ratio_7,e_ratio_8,e_ratio_9,e_ratio_10,e_ratio_11,e_ratio_12,e_ratio_13,e_ratio_14,e_ratio_15,e_ratio_16,e_ratio_17,e_ratio_18,e_ratio_19,e_ratio_20,e_ratio_21,e_ratio_22,e_ratio_23,e_ratio_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,e_ratio_avg,pct_price_diff_avg,e_ratio_rank,price_rank
97,PriceChange_LinearReg,20,8,0,1.3156,1.4694,1.4686,1.5552,1.5828,1.5958,1.5895,1.5700,1.5646,1.5419,1.5469,1.5504,1.5492,1.5910,1.5941,1.5832,1.5788,1.5609,1.5316,1.5156,1.5087,1.4971,1.4805,1.4783,0.73,0.78,0.48,0.37,0.40,0.37,0.32,0.28,0.28,0.20,0.23,0.18,0.0,0.00,-0.05,-0.24,-0.23,-0.39,-0.51,-0.70,-0.65,-0.77,-0.85,-0.96,347,1.534154,-0.030417,185.351496,-10.554583
99,PriceChange_LinearReg,20,12,0,1.3128,1.4685,1.4688,1.5508,1.5738,1.5869,1.5811,1.5621,1.5570,1.5346,1.5398,1.5435,1.5426,1.5847,1.5902,1.5796,1.5755,1.5576,1.5284,1.5123,1.5055,1.4939,1.4774,1.4752,0.72,0.78,0.49,0.39,0.38,0.34,0.31,0.26,0.26,0.18,0.19,0.18,0.0,0.00,-0.12,-0.25,-0.26,-0.42,-0.61,-0.71,-0.71,-0.79,-0.88,-0.98,344,1.529275,-0.052083,182.070600,-17.916667
98,PriceChange_LinearReg,20,10,0,1.3172,1.4724,1.4719,1.5528,1.5750,1.5881,1.5822,1.5631,1.5579,1.5354,1.5406,1.5443,1.5433,1.5855,1.5888,1.5778,1.5735,1.5557,1.5264,1.5104,1.5036,1.4921,1.4756,1.4734,0.73,0.78,0.50,0.37,0.38,0.34,0.30,0.26,0.24,0.16,0.15,0.18,0.0,0.00,-0.14,-0.25,-0.27,-0.43,-0.62,-0.72,-0.72,-0.79,-0.89,-0.98,343,1.529458,-0.059167,181.604208,-20.294167
96,PriceChange_LinearReg,20,6,0,1.2959,1.4287,1.4339,1.5158,1.5516,1.5696,1.5790,1.5602,1.5543,1.5326,1.5378,1.5416,1.5407,1.5829,1.5798,1.5696,1.5716,1.5550,1.5283,1.5070,1.4995,1.4827,1.4686,1.4697,0.73,0.77,0.48,0.35,0.39,0.36,0.32,0.26,0.26,0.20,0.19,0.17,0.0,0.00,-0.12,-0.25,-0.26,-0.42,-0.61,-0.71,-0.72,-0.78,-0.86,-0.97,344,1.519017,-0.050833,178.541733,-17.486667
95,PriceChange_LinearReg,20,4,0,1.2913,1.4269,1.4336,1.5235,1.5585,1.5762,1.5697,1.5517,1.5461,1.5298,1.5359,1.5398,1.5389,1.5815,1.5784,1.5671,1.5684,1.5517,1.5247,1.5032,1.4956,1.4787,1.4640,1.4649,0.73,0.78,0.49,0.36,0.39,0.36,0.32,0.27,0.27,0.22,0.23,0.18,0.0,0.00,-0.12,-0.25,-0.26,-0.44,-0.62,-0.72,-0.75,-0.79,-0.88,-0.98,342,1.516671,-0.050417,176.701425,-17.242500
87,PriceChange_LinearReg,18,8,0,1.3305,1.4841,1.5081,1.6012,1.5971,1.6082,1.6118,1.5863,1.5566,1.5294,1.5305,1.5384,1.5382,1.5854,1.5888,1.5744,1.5695,1.5541,1.5264,1.5108,1.4987,1.4878,1.4716,1.4756,0.74,0.82,0.52,0.42,0.38,0.35,0.32,0.24,0.20,0.14,0.13,0.12,0.0,0.00,-0.14,-0.27,-0.25,-0.49,-0.64,-0.76,-0.80,-0.80,-0.90,-1.00,321,1.535979,-0.069583,172.049313,-22.336250
92,PriceChange_LinearReg,19,8,0,1.2991,1.4642,1.4700,1.5500,1.5581,1.5751,1.5686,1.5501,1.5227,1.4983,1.5023,1.5043,1.5050,1.5494,1.5558,1.5454,1.5422,1.5277,1.5021,1.4878,1.4818,1.4717,1.4553,1.4536,0.73,0.82,0.49,0.37,0.39,0.36,0.32,0.28,0.26,0.18,0.14,0.17,0.0,0.00,-0.05,-0.23,-0.23,-0.40,-0.56,-0.70,-0.68,-0.77,-0.84,-0.93,336,1.505858,-0.036667,169.968400,-12.320000
89,PriceChange_LinearReg,18,12,0,1.3275,1.4832,1.5084,1.5966,1.5872,1.5985,1.6028,1.5777,1.5483,1.5213,1.5228,1.5308,1.5310,1.5785,1.5844,1.5704,1.5659,1.5505,1.5228,1.5073,1.4952,1.4843,1.4683,1.4724,0.75,0.83,0.52,0.44,0.36,0.34,0.30,0.21,0.16,0.08,0.12,0.15,0.0,0.00,-0.17,-0.29,-0.27,-0.52,-0.69,-0.78,-0.85,-0.84,-0.95,-1.06,318,1.530671,-0.090000,168.753325,-28.620000
88,PriceChange_LinearReg,18,10,0,1.3324,1.4875,1.5121,1.5990,1.5886,1.5999,1.6040,1.5788,1.5492,1.5222,1.5236,1.5317,1.5318,1.5794,1.5829,1.5684,1.5637,1.5484,1.5207,1.5052,1.49

# Save new config data to config file

In [10]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [20]}, 
              'LinearReg': {'timeperiod': [12], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,48.911181,0.698576,-44.928403,0.749826,33
2,53.224514,1.224826,-96.531319,0.555521,60
4,63.002222,1.507500,-106.731771,3.426736,123
3,57.661771,1.482188,-108.683125,1.100035,90
5,63.067674,1.100035,-111.523819,7.445278,154
6,63.261736,1.080729,-129.090833,7.653299,176
7,59.737639,0.319965,-155.473542,7.016875,216
8,60.148194,0.825590,-160.278958,10.196632,242
9,60.661910,0.429826,-167.830035,11.735139,278
